# Домашнее задание
## Параллельные вычисления

Тема: "Решение задачи линейного программирования при помощи параллельного варианта симплекс-метода"

Выполнили:
- Зимин Григорий Сергеевич, ИУ8-112
- Александров Алексей Н., ИУ8-114
- Сакулин Даниил Игоревич, ИУ8-115

In [12]:
import logging
from pathlib import Path

import numpy as np
import cupy as cp

from src.simplex.simplex_problem import SimplexProblem, Solution

In [13]:
logging.basicConfig(level=logging.INFO, format='%(message)s')

In [14]:
sample_input_path = Path("sample_input_LLP.json")
problem = SimplexProblem(sample_input_path)

In [15]:
solution, _ = problem.solve()
solution

([0.013888888888888881, 0.125, 0, 0], 0.1388888888888889)

In [16]:
problem = SimplexProblem(sample_input_path, use_gpu="cupy", verbose=False)

In [17]:
solution, _ = problem.solve(timer=True)
solution

([0.013888888888888881, 0.125, 0, 0], 0.1388888888888889)

### Warming Up!

In [7]:
a_np = np.random.rand(10000, 10000)
b_np = np.random.rand(10000, 10000)

In [8]:
%%time
c = np.dot(a_np, b_np)
c

CPU times: user 2min 33s, sys: 3.89 s, total: 2min 37s
Wall time: 13.5 s


array([[2526.90698924, 2524.86463316, 2501.39075187, ..., 2532.18647524,
        2510.12956302, 2503.58736248],
       [2549.89898054, 2540.10875159, 2511.16575564, ..., 2549.45427264,
        2532.00573068, 2519.97893658],
       [2520.30518419, 2506.07217727, 2481.06753459, ..., 2517.0552054 ,
        2489.77561285, 2483.93761519],
       ...,
       [2527.36251624, 2523.61092914, 2483.56758114, ..., 2531.75961199,
        2517.23756051, 2490.31150528],
       [2537.55285633, 2517.96527258, 2503.00362948, ..., 2535.87954506,
        2518.14377282, 2509.47616364],
       [2552.46961591, 2543.88741788, 2516.77699398, ..., 2550.76795737,
        2547.79497279, 2518.10768312]])

In [9]:
%%time
a_cp = cp.asarray(a_np)
b_cp = cp.asarray(b_np)
c = cp.dot(a_cp, b_cp)
c

CPU times: user 252 ms, sys: 2.21 s, total: 2.46 s
Wall time: 1.8 s


array([[2526.90698924, 2524.86463316, 2501.39075187, ..., 2532.18647524,
        2510.12956302, 2503.58736248],
       [2549.89898054, 2540.10875159, 2511.16575564, ..., 2549.45427264,
        2532.00573068, 2519.97893658],
       [2520.30518419, 2506.07217727, 2481.06753459, ..., 2517.0552054 ,
        2489.77561285, 2483.93761519],
       ...,
       [2527.36251624, 2523.61092914, 2483.56758114, ..., 2531.75961199,
        2517.23756051, 2490.31150528],
       [2537.55285633, 2517.96527258, 2503.00362948, ..., 2535.87954506,
        2518.14377282, 2509.47616364],
       [2552.46961591, 2543.88741788, 2516.77699398, ..., 2550.76795737,
        2547.79497279, 2518.10768312]])

### Проведение бенчмарка

In [10]:
logging.basicConfig(level=logging.CRITICAL, format='%(message)s')

In [11]:
from src.benchmark import benchmark_lp_solver

increase_iter = (5, 10, 20, 40, 80, 160, 320)

benchmark_lp_solver(
    problem_size_iterable=increase_iter,
)

Размерность задачи (число переменных / ограничений),CPU Time (ms),GPU Time (ms),Прирост производительности (?)
5,11.255,7.434,1.51
10,17.538,10.483,1.67
20,81.589,23.707,3.44
40,418.501,57.056,7.33
80,1449.153,96.853,14.96
160,25407.873,844.248,30.10
320,50728.075,834.236,60.81
